In [10]:
# -*- coding: utf-8 -*-
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"]='expandable_segments>True'

import torch
import os.path as osp
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import functools

In [11]:
def make_layer(block, n_layers):
    """Creates a sequential layer composed of multiple blocks.

    Args:
        block (nn.Module): The block module to be repeated.
        n_layers (int): The number of times to repeat the block.

    Returns:
        nn.Sequential: A sequential container of the repeated blocks.
    """
    layers = []
    for _ in range(n_layers):
        layers.append(block())
    return nn.Sequential(*layers)

In [12]:
class ResidualDenseBlock_5C(nn.Module):
    """Residual Dense Block with 5 convolutional layers.

    Args:
        nf (int): Number of input features. Default: 64.
        gc (int): Growth channel, i.e., intermediate channels. Default: 32.
        bias (bool): Whether to use bias in convolutional layers. Default: True.
    """
    def __init__(self, nf=64, gc=32, bias=True):
        super(ResidualDenseBlock_5C, self).__init__()
        # gc: growth channel, i.e. intermediate channels
        self.conv1 = nn.Conv2d(nf, gc, 3, 1, 1, bias=bias)
        self.conv2 = nn.Conv2d(nf + gc, gc, 3, 1, 1, bias=bias)
        self.conv3 = nn.Conv2d(nf + 2 * gc, gc, 3, 1, 1, bias=bias)
        self.conv4 = nn.Conv2d(nf + 3 * gc, gc, 3, 1, 1, bias=bias)
        self.conv5 = nn.Conv2d(nf + 4 * gc, nf, 3, 1, 1, bias=bias)
        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

        # Initialization (optional, can be added if specific weight initialization is needed)
        # mutil.initialize_weights([self.conv1, self.conv2, self.conv3, self.conv4, self.conv5], 0.1)

    def forward(self, x):
        """Forward pass through the Residual Dense Block."""
        x1 = self.lrelu(self.conv1(x))
        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))
        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))
        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        # Residual scaling and addition
        return x5 * 0.2 + x

In [13]:
class RRDB(nn.Module):
    """Residual in Residual Dense Block. Contains multiple RDBs.

    Args:
        nf (int): Number of input features.
        gc (int): Growth channel. Default: 32.
    """
    def __init__(self, nf, gc=32):
        super(RRDB, self).__init__()
        self.RDB1 = ResidualDenseBlock_5C(nf, gc)
        self.RDB2 = ResidualDenseBlock_5C(nf, gc)
        self.RDB3 = ResidualDenseBlock_5C(nf, gc)

    def forward(self, x):
        """Forward pass through the RRDB."""
        out = self.RDB1(x)
        out = self.RDB2(out)
        out = self.RDB3(out)
        # Residual scaling and addition
        return out * 0.2 + x


In [14]:
class RRDBNet(nn.Module):
    """Residual in Residual Dense Block Network (ESRGAN).

    Args:
        in_nc (int): Number of input image channels.
        out_nc (int): Number of output image channels.
        nf (int): Number of features in intermediate layers.
        nb (int): Number of RRDB blocks.
        gc (int): Growth channel in RDBs. Default: 32.
    """
    def __init__(self, in_nc, out_nc, nf, nb, gc=32):
        super(RRDBNet, self).__init__()
        # Create a factory function for RRDB blocks with specified nf and gc
        RRDB_block_f = functools.partial(RRDB, nf=nf, gc=gc)

        # Initial convolution layer
        self.conv_first = nn.Conv2d(in_nc, nf, 3, 1, 1, bias=True)

        # Main trunk of RRDB blocks
        self.RRDB_trunk = make_layer(RRDB_block_f, nb)
        self.trunk_conv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)

        # Upsampling layers
        self.upconv1 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.upconv2 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)

        # High-resolution convolution and final output convolution
        self.HRconv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.conv_last = nn.Conv2d(nf, out_nc, 3, 1, 1, bias=True)

        # Activation function
        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

    def forward(self, x):
        """Forward pass through the RRDBNet."""
        # Initial feature extraction
        fea = self.conv_first(x)
        # Pass through RRDB trunk and add residual connection
        trunk = self.trunk_conv(self.RRDB_trunk(fea))
        fea = fea + trunk

        # Upsampling and activation
        fea = self.lrelu(self.upconv1(F.interpolate(fea, scale_factor=2, mode='nearest')))
        fea = self.lrelu(self.upconv2(F.interpolate(fea, scale_factor=2, mode='nearest')))

        # Final convolutions to produce output image
        out = self.conv_last(self.lrelu(self.HRconv(fea)))

        return out


# ---------------------------------------------------------------------------- #
#                            Main Execution Logic                            #
# ---------------------------------------------------------------------------- #

In [15]:
# ---------------------------------------------------------------------------- #
#                            Main Execution Logic                            #
# ---------------------------------------------------------------------------- #

In [16]:
# --- Configuration ---
model_path = '/kaggle/input/rrdb_esrgan_x4.pth/pytorch/default/1/RRDB_ESRGAN_x4.pth'  # Path to the pre-trained model weights
# Use GPU if available, otherwise fallback to CPU
device = torch.device('cpu')
# device = torch.device('cpu') # Uncomment this line to force CPU usage

test_img_folder = '/kaggle/input/new-code/LR/*' # Folder containing low-resolution images (adjust path as needed)
results_folder = 'results' # Folder to save the high-resolution output images

# Create results folder if it doesn't exist
if not osp.exists(results_folder):
    os.makedirs(results_folder)
    print(f"Created results folder: {results_folder}")

In [17]:
# --- Model Definition and Loading ---
# Initialize the RRDBNet model. Parameters should match the trained model:
# RRDBNet(in_nc=3, out_nc=3, nf=64, nb=23, gc=32)
model = RRDBNet(in_nc=3, out_nc=3, nf=64, nb=23, gc=32)

# Load the pre-trained weights
print(f"Loading model from: {model_path}")
model.load_state_dict(torch.load(model_path), strict=True)

# Set the model to evaluation mode (disables dropout, batch norm updates, etc.)
model.eval()

# Move the model to the specified device (GPU or CPU)
model = model.to(device)

print(f"Model loaded successfully. Using device: {device}")
print('Testing...')

Loading model from: /kaggle/input/rrdb_esrgan_x4.pth/pytorch/default/1/RRDB_ESRGAN_x4.pth
Model loaded successfully. Using device: cpu
Testing...


/tmp/ipykernel_76/2769069695.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path), strict=True)


In [28]:
from torchsummary import summary

# Print model summary (assuming input images are 3x96x96)
summary(model, input_size=(3, 96, 96), device=str(device))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 96, 96]           1,792
            Conv2d-2           [-1, 32, 96, 96]          18,464
         LeakyReLU-3           [-1, 32, 96, 96]               0
            Conv2d-4           [-1, 32, 96, 96]          27,680
         LeakyReLU-5           [-1, 32, 96, 96]               0
            Conv2d-6           [-1, 32, 96, 96]          36,896
         LeakyReLU-7           [-1, 32, 96, 96]               0
            Conv2d-8           [-1, 32, 96, 96]          46,112
         LeakyReLU-9           [-1, 32, 96, 96]               0
           Conv2d-10           [-1, 64, 96, 96]         110,656
ResidualDenseBlock_5C-11           [-1, 64, 96, 96]               0
           Conv2d-12           [-1, 32, 96, 96]          18,464
        LeakyReLU-13           [-1, 32, 96, 96]               0
           Conv2d-14           [-1,

In [26]:
# model.__getattr__

In [27]:
# model.summary()

In [ ]:
# --- Image Processing Loop ---
idx = 0
image_paths = glob.glob(test_img_folder)

if not image_paths:
    print(f"Error: No images found in {test_img_folder}. Please check the path.")
else:
    print(f"Found {len(image_paths)} images to process.")

    for path in image_paths:
        idx += 1
        base = osp.splitext(osp.basename(path))[0] # Get filename without extension
        print(f"Processing image {idx}: {base}")

        # Read image using OpenCV
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if img is None:
            print(f"Warning: Could not read image {path}. Skipping.")
            continue

        # Preprocess the image: normalize, convert to tensor, rearrange dimensions
        img = img * 1.0 / 255.0 # Normalize pixel values to [0, 1]
        img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float() # HWC -> CHW, BGR -> RGB, Convert to float tensor
        img_LR = img.unsqueeze(0) # Add batch dimension (BCHW)
        img_LR = img_LR.to(device) # Move the input tensor to the device

        # Perform inference
        with torch.no_grad(): # Disable gradient calculation for efficiency
            output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy() # Get output, remove batch dim, move to CPU, clamp, convert to numpy

        # Postprocess the output image: rearrange dimensions, denormalize
        output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0)) # CHW -> HWC, RGB -> BGR
        output = (output * 255.0).round() # Denormalize pixel values to [0, 255] and round

        # Save the resulting high-resolution image
        output_path = osp.join(results_folder, f'{base}_rlt.png')
        cv2.imwrite(output_path, output)
        print(f"Saved result to: {output_path}")

print("Processing complete.")


# For downloading the Result Images

In [4]:
import shutil

# Zip the folder
shutil.make_archive('results', 'zip', 'results')


'/kaggle/working/results.zip'

In [5]:
from IPython.display import FileLink

# Create a clickable download link
FileLink('results.zip')


/kaggle/working/results.zip